In [2]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Quiescence Search

In [3]:
using Pkg
# Pkg.add("Chess")
using Chess
using NBInclude

[Chess Programming Wiki "Quiescence Search"](https://www.chessprogramming.org/Quiescence_Search)

In [4]:
@nbinclude("Memoization.ipynb")

updateBoardData (generic function with 1 method)

## Auxiliary functions

### Function: hasCaptureMoves
This function takes in a chess game state and returns a boolean indicating whether there are any capture moves available for the opponent.

Arguments:
1. `State::Board:` A chess board in the current state.

Returns:
1. `true`: if there are any capture moves available for the opponent, `false` otherwise.

In [2]:
function hasCaptureMoves(State)
    allPiecesSq = pieces(State, coloropp(sidetomove(State)))
    return any(isattacked(State, square, coloropp(sidetomove(State))) for square in allPiecesSq)
end

hasCaptureMoves (generic function with 1 method)

In [6]:
function isTacticalMove(State, move)
    if pieceon(State, to(move)) != EMPTY || ispromotion(move) || epsquare(State) == to(move)
        return true
    end
    undoinfo = domove!(State, move)
    isNextMoveCheck = ischeck(State)
    undomove!(State, undoinfo)
    
    return isNextMoveCheck
end 

isTacticalMove (generic function with 1 method)

In [7]:
function isQuiet(State)
    return any(isTacticalMove(State, move) for move in moves(State))

end

isQuiet (generic function with 1 method)

## no Quiescence Search

In [8]:
function noQuiesce(State, score)
    return score
end

noQuiesce (generic function with 1 method)

## General Quiescence Search

In [9]:
function quiesceNoPrune(State, score, hash)
    bestscore = score
    for move in moves(State)
        if isTacticalMove(State, move)
            nextScore, nextHash = updateBoardData(State, score, hash, move)
            # nextBoard = domove(State, move)
            undoinfo = domove!(State, move)
            # value = -evaluate(State, quiesce, nextScore, nextHash, depth-1, -beta, -alpha)
            value = quiesceNoPrune(State, nextScore, nextHash)
            undomove!(State, undoinfo)
            bestscore = sidetomove(State) == WHITE ? max(bestscore, value) : min(bestscore, value)
        end
    end
    
    return bestscore
end

quiesceNoPrune (generic function with 1 method)

In [10]:
function quiesce(State, score, hash, depth, alpha, beta)
    display(State)
    println("$(score), $(depth), $(alpha), $(beta)")
    value = score
    if value >= beta
        println("Pruned 1")
        return beta
    end
    if value > alpha
        alpha = value
    end
    for move in moves(State)
        if isTacticalMove(State, move)
            nextScore, nextHash = updateBoardData(State, score, hash, move)
            
            undoinfo = domove!(State, move)
            
            # value = -evaluate(State, quiesce, nextScore, nextHash, depth-1, -beta, -alpha)
            value = -quiesce(State, nextScore, nextHash, depth-1, -beta, -alpha)
            
            println(value)
            undomove!(State, undoinfo)
            
            if value >= beta
                return beta
            end
            if value > alpha
                alpha = value
            end
        end
    end
    
    return alpha
end

quiesce (generic function with 1 method)

## Quiescence Min and Max

In [11]:
function quiesceMax(State, score, hash, depth, alpha, beta)
    value = score
    if value >= beta
        return beta
    end
    if value > alpha
        alpha = value
    end
    for move in moves(State)
        if isTacticalMove(State, move)
            nextScore, nextHash = updateBoardData(State, score, hash, move)
            
            undoinfo = domove!(State, move)
            # value = -evaluate(State, quiesce, nextScore, nextHash, depth-1, -beta, -alpha)
            value = quiesceMin(State, nextScore, nextHash, depth-1, alpha, beta)
            undomove!(State, undoinfo)
            
            if value >= beta
                return beta
            end
            if value > alpha
                alpha = value
            end
        end
    end
    
    return alpha
end

quiesceMax (generic function with 1 method)

In [12]:
function quiesceMin(State, score, hash, depth, alpha, beta)
    value = score
    if value <= alpha
        return alpha
    end
    if value < beta
        beta = value
    end
    for move in moves(State)
        if isTacticalMove(State, move)
            nextScore, nextHash = updateBoardData(State, score, hash, move)
            
            undoinfo = domove!(State, move)
            # value = -evaluate(State, quiesce, nextScore, nextHash, depth-1, -beta, -alpha)
            value = quiesceMax(State, nextScore, nextHash, depth-1, alpha, beta)
            undomove!(State, undoinfo)
            
            if value <= alpha
                return alpha
            end
            if value < beta
                beta = value
            end
        end
    end
    
    return beta
end

quiesceMin (generic function with 1 method)

## Quiescence Search with see() function

In [13]:
function quiesceSee(State, score, alpha, beta)
    value = score
    if value >= beta
        return beta
    end
    if value > alpha
        alpha = value
    end
    if sidetomove(State) == WHITE
        bestEstimate = -Inf
        for move in moves(State)
            bestEstimate = max(bestEstimate, see(State, move))
        end
    else 
        bestEstimate = Inf
        for move in moves(State)
            bestEstimate = min(bestEstimate, see(State, move))
        end
    end

    return value + bestEstimate * 100
end

quiesceSee (generic function with 1 method)

In [14]:
function quiesceSee(State, score, alpha, beta)
    value = score
    if value >= beta
        return beta
    end
    if value > alpha
        alpha = value
    end
    if sidetomove(State) == WHITE
        bestEstimate = -Inf
        for move in moves(State)
            bestEstimate = max(bestEstimate, see(State, move))
        end
    else 
        bestEstimate = Inf
        for move in moves(State)
            bestEstimate = min(bestEstimate, see(State, move))
        end
    end

    return value + bestEstimate * 100
end

quiesceSee (generic function with 1 method)

## Strategic Quiescence Search

In [15]:
function evalPlus(State)
    if isCheck(State)
        return Inf
    end
    if hasCaptureMoves(State) && ispromotion(State)
    end
end

evalPlus (generic function with 1 method)

In [16]:
function strategicQS(State, score, hash, alpha, beta)
    value = score
    if value >= beta
        return beta
    end
    if value > alpha
        alpha = value
    end
    
    for move in moves(State)
        nextScore, nextHash = updateBoardData(State, score, hash, move)
        undoinfo = domove!(State, move)
        evalPlusValue = evalPlus(State)
        if evalPlusValue > alpha
            actVal = -strategicQS(State, nextScore, nextHash, -beta, -alpha)
            if actVal > value
                value = actVal
                if value >= beta
                    return beta
                end
                if value > alpha
                    alpha = value
                end
            end
        elseif evalPlusValue > value
            value = evalPlusValue
        end
        undomove!(State, move)
    end
    return value
end

strategicQS (generic function with 1 method)